## Simulate data

This script is used to generate simulated data. Then we can run parameter_recovery to estimate those parameters from the simulated data


In [9]:
# Built-in/Generic Imports
import os

# Libs
import numpy as np
import pandas as pd
from scipy.stats import bernoulli

In [10]:

### Defining the grids, and functions to make sets as dataframes: choice_set,param_set,response_set

def set_grids(nb_kappa=10,nb_gamma=10):
    grid_design = {
        # now: time_null = 0
        'cdd_immed_wait': [0],
        # time to get reward (delay_wait)
        'cdd_delay_wait': [2, 7, 15, 36, 72, 108, 144, 180],
        # immediate reward from experiment: $2, $10, $20 ... fixed to $10
        'cdd_immed_amt': [10, 20],
        # reward amount set to vary according to experiment
        'cdd_delay_amt': [7, 14, 21, 30, 41, 50, 65]
    }
    grid_param = {
        # 50 points on [10^-5, ..., 1] in a log scale
        'kappa_gt': np.logspace(-5, 0, nb_kappa, base=10),
        # 10 points on (0, 5] in a linear scale
        'gamma_gt': np.linspace(0, 5, nb_gamma+1)[1:]
    }
    return grid_design,grid_param

def insert_var(grid=[],var_nb=0,list_var=[]):
    if not var_nb:
        grid = list_var
    elif var_nb==1:
        grid = [[g]+[i] for g in grid for i in list_var]
    else:
        grid = [g+[i] for g in grid for i in list_var]
    return grid

def make_grid(grid_values):
    grid = []
    labels = grid_values.keys()
    for var_nb,row in enumerate(grid_values.items()):
        grid = insert_var(grid=grid,var_nb=var_nb,list_var=row[1])
    grid_df = pd.DataFrame(grid,columns=labels)
    return grid_df


In [11]:

def simulate_response(design,PARAM_TRUE):
    # Calculate the probability to choose a variable option
    tn, tr, vn, vr = (design['cdd_immed_wait'], design['cdd_delay_wait'],design['cdd_immed_amt'], design['cdd_delay_amt'])
    kappa, gamma = PARAM_TRUE['kappa_gt'], PARAM_TRUE['gamma_gt']
    
    SV_null = vn * (1. / (1 + kappa * tn))
    SV_reward = vr * (1. / (1 + kappa * tr))
    p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))

    # Randomly sample a binary choice response from Bernoulli distribution
    return bernoulli.rvs(p_obs)

def generate_response(design_df,PARAM_TRUE):
    response= []
    df = pd.DataFrame([])
    for index, design in design_df.iterrows():
        response += [simulate_response(design,PARAM_TRUE)]
    df['cdd_choice'] = response
    df['cdd_trial_type'] = 'task'
    return df


In [13]:
# can set resolution (number of parameter) for kappa and gamma
grid_design,grid_param = set_grids(nb_kappa=3,nb_gamma=3)

design_df = make_grid(grid_design)
fn = os.path.join('simul','design_set.csv')
print('Saving design to : {}'.format(fn))
design_df.to_csv(fn)

param_df = make_grid(grid_param)
fn = os.path.join('simul','ground_truth.csv')
print('Saving ground truth parameters to : {}'.format(fn))
param_df.to_csv(fn)

for index, PARAM_TRUE in param_df.iterrows():
    response_df = generate_response(design_df,PARAM_TRUE)
    subj_dir = os.path.join('simul','split','p{0:04d}'.format(index),'cdd')
    if not os.path.exists(subj_dir):
        os.makedirs(subj_dir)
    fn = os.path.join(subj_dir,'p{0:04d}_cdd.csv'.format(index))
    print('Saving response to : {}'.format(fn))
    df = pd.concat([design_df,response_df],axis=1)
    df.to_csv(fn)
df


Saving design to : simul/design_set.csv
Saving ground truth parameters to : simul/ground_truth.csv
Saving response to : simul/split/p0000/cdd/p0000_cdd.csv
Saving response to : simul/split/p0001/cdd/p0001_cdd.csv
Saving response to : simul/split/p0002/cdd/p0002_cdd.csv
Saving response to : simul/split/p0003/cdd/p0003_cdd.csv
Saving response to : simul/split/p0004/cdd/p0004_cdd.csv
Saving response to : simul/split/p0005/cdd/p0005_cdd.csv
Saving response to : simul/split/p0006/cdd/p0006_cdd.csv
Saving response to : simul/split/p0007/cdd/p0007_cdd.csv
Saving response to : simul/split/p0008/cdd/p0008_cdd.csv


,cdd_immed_wait,cdd_delay_wait,cdd_immed_amt,cdd_delay_amt,cdd_choice,cdd_trial_type
0,0.0,2.0,10.0,7.0,NaN,NaN
1,0.0,2.0,10.0,14.0,NaN,NaN
2,0.0,2.0,10.0,21.0,NaN,NaN
3,0.0,2.0,10.0,30.0,NaN,NaN
4,0.0,2.0,10.0,41.0,NaN,NaN
...,...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,0.0,task
108,NaN,NaN,NaN,NaN,0.0,task
109,NaN,NaN,NaN,NaN,0.0,task
110,NaN,NaN,NaN,NaN,0.0,task
